In [1]:
from inaFaceGender import GenderVideo
from inaFaceGender.face_detector import LibFaceDetection
from inaFaceGender.opencv_utils import get_video_properties, analysisFPS2subsamp_coeff
import os
import pandas as pd

In [2]:
gv = GenderVideo(face_detector=LibFaceDetection())

2021-10-31 15:33:02.320449: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-31 15:33:02.321829: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-31 15:33:02.322064: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-31 15:33:02.322368: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
analysis_fps = 5
vid_src = './media/pexels-artem-podrez-5725953.mp4'
%time ret = gv(vid_src, fps=analysis_fps)

2021-10-31 15:33:07.112554: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-10-31 15:33:07.729496: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8204
2021-10-31 15:33:07.734024: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-10-31 15:33:09.116905: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-10-31 15:33:09.116982: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were 

CPU times: user 12 s, sys: 764 ms, total: 12.8 s
Wall time: 9.3 s


In [4]:
ret

,frame,bbox,face_detect_conf,sex_decfunc,age_decfunc,sex_label,age_label
0,0,"(945, -16, 1138, 176)",0.999999,8.180841,3.883812,m,33.838120
1,0,"(70, 118, 272, 320)",0.999996,-12.277651,3.165739,f,26.657391
2,0,"(310, 163, 491, 343)",0.999979,-10.850054,2.996130,f,24.961305
3,0,"(558, 201, 727, 371)",0.999974,8.937360,2.911818,m,24.118180
4,0,"(744, 22, 929, 207)",0.981539,-7.614778,2.310285,f,18.102846
...,...,...,...,...,...,...,...
219,348,"(343, 130, 472, 259)",0.999986,-8.917940,2.868898,f,23.688982
220,348,"(-44, 108, 165, 319)",0.999736,-7.542460,3.348511,f,28.485115
221,354,"(877, 65, 1013, 201)",0.999999,-8.295111,2.623810,f,21.238103
222,354,"(611, 260, 796, 444)",0.999990,12.288671,2.911372,m,24.113722


In [5]:
import datetime
def sec2hmsms(s):
    td = datetime.timedelta(seconds=s)
    h,m,s = str(td).split(':')
    return '%d:%d:%.2f' % (int(h), int(m), float(s))
#    if len(ret) == 7:
#        return ret + '.00'
#    assert len(ret) == 14, ret
#    return ret[:-4]
sec2hmsms(3600+66.6), sec2hmsms(.06), sec2hmsms(.0006), sec2hmsms(1.)

('1:1:6.60', '0:0:0.06', '0:0:0.00', '0:0:1.00')

In [6]:
def analysis2displaydf(df, fps, subsamp_coeff):
    ret = pd.DataFrame()
    ret['frame'] = df.frame
    ret['bbox'] = df.bbox
    ret[['x1', 'y1', 'x2', 'y2']] = df.apply(lambda x: x.bbox, axis=1, result_type="expand")
    ret['rgb_color'] = df.sex_label.map(lambda x: '0000FF' if x == 'm' else '00FF00')
    ret['bgr_color'] = ret.rgb_color.map(lambda x: x[4:] + x[2:4] + x[:2])
    ret['start'] = df.frame.map(lambda x: sec2hmsms(x / fps))
    ret['stop'] = df.frame.map(lambda x: sec2hmsms((x + subsamp_coeff) / fps))
    ret['text'] = df.apply(lambda x: 'sex: %s (%.1f) - age: %.1f' % (x.sex_label, x.sex_decfunc, x.age_label), axis=1)
    return ret

video_props = get_video_properties(vid_src)
fps, width, height = [video_props[e] for e in ['fps', 'width', 'height']]
if analysis_fps is None:
    subsamp_coeff = 1
else:
    subsamp_coeff = analysisFPS2subsamp_coeff(vid_src, analysis_fps)
displaydf = analysis2displaydf(ret, fps, subsamp_coeff)
displaydf

,frame,bbox,x1,y1,x2,y2,rgb_color,bgr_color,start,stop,text
0,0,"(945, -16, 1138, 176)",945,-16,1138,176,0000FF,FF0000,0:0:0.00,0:0:0.20,sex: m (8.2) - age: 33.8
1,0,"(70, 118, 272, 320)",70,118,272,320,00FF00,00FF00,0:0:0.00,0:0:0.20,sex: f (-12.3) - age: 26.7
2,0,"(310, 163, 491, 343)",310,163,491,343,00FF00,00FF00,0:0:0.00,0:0:0.20,sex: f (-10.9) - age: 25.0
3,0,"(558, 201, 727, 371)",558,201,727,371,0000FF,FF0000,0:0:0.00,0:0:0.20,sex: m (8.9) - age: 24.1
4,0,"(744, 22, 929, 207)",744,22,929,207,00FF00,00FF00,0:0:0.00,0:0:0.20,sex: f (-7.6) - age: 18.1
...,...,...,...,...,...,...,...,...,...,...,...
219,348,"(343, 130, 472, 259)",343,130,472,259,00FF00,00FF00,0:0:11.60,0:0:11.80,sex: f (-8.9) - age: 23.7
220,348,"(-44, 108, 165, 319)",-44,108,165,319,00FF00,00FF00,0:0:11.60,0:0:11.80,sex: f (-7.5) - age: 28.5
221,354,"(877, 65, 1013, 201)",877,65,1013,201,00FF00,00FF00,0:0:11.80,0:0:12.00,sex: f (-8.3) - age: 21.2
222,354,"(611, 260, 796, 444)",611,260,796,444,0000FF,FF0000,0:0:11.80,0:0:12.00,sex: m (12.3) - age: 24.1


In [7]:
from Cheetah.Template import Template


In [9]:
t = Template(file='./inaFaceGender/template.ass')

In [10]:
t.height = height
t.width = width
t.display_df = displaydf

In [11]:
with open('cheetahtest.ass', 'wt') as fid:
    print(t, file=fid)

In [12]:
os.system('vlc %s --sub-file ./cheetahtest.ass' % vid_src)

[00005618219d8670] main libvlc: Running vlc with the default interface. Use 'cvlc' to use vlc without interface.
[ass] Shaper: FriBidi 0.19.7 (SIMPLE) HarfBuzz-ng 2.6.4 (COMPLEX)
[ass] Using font provider fontconfig
libva info: VA-API version 1.7.0
libva info: Trying to open /usr/lib/x86_64-linux-gnu/dri/nvidia_drv_video.so
libva info: va_openDriver() returns -1
[00007fe77c003880] glconv_vaapi_x11 gl error: vaInitialize: unknown libva error
libva info: VA-API version 1.7.0
libva info: Trying to open /usr/lib/x86_64-linux-gnu/dri/iHD_drv_video.so
libva info: Found init function __vaDriverInit_1_7
libva info: va_openDriver() returns 0
[00007fe788060a30] main video output error: video output creation failed
[00007fe794c17dd0] main decoder error: failed to create video output
[00007fe794c17dd0] avcodec decoder: Using NVIDIA VDPAU Driver Shared Library  470.63.01  Tue Aug  3 20:29:27 UTC 2021 for hardware decoding
[ass] fontselect: (DejaVu Sans, 400, 0) -> /usr/share/fonts/truetype/dejavu/D

0